<a href="https://colab.research.google.com/github/worawit-saetan/Kaggle-Dataset---Spaceship-Titanic/blob/main/Kaggle_Dataset_Spaceship_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle Dataset - Spaceship Titanic 

Predict which passengers are transported to an alternate dimension

## The outline of this project

drop some unnecessary columns

+ fill in missing data

+ encode the data

+ train XGBoost model with GridSearchCV on training set

+ train another XGBoost model with best parameter setting (from gridsearch) on the merged training and test set

In [1]:
import pandas as pd
dataset = pd.read_csv('/content/train.csv')
dataset

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [2]:
dataset.shape


(8693, 14)

In [3]:
X = dataset.drop(['Transported'],axis=1)

In [4]:
y = dataset['Transported']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(6954, 13)
(1739, 13)
(6954,)
(1739,)


Check null values

In [6]:
X_train.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6954 entries, 1085 to 2037
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   6954 non-null   object 
 1   HomePlanet    6793 non-null   object 
 2   CryoSleep     6784 non-null   object 
 3   Cabin         6798 non-null   object 
 4   Destination   6815 non-null   object 
 5   Age           6811 non-null   float64
 6   VIP           6794 non-null   object 
 7   RoomService   6809 non-null   float64
 8   FoodCourt     6808 non-null   float64
 9   ShoppingMall  6791 non-null   float64
 10  Spa           6810 non-null   float64
 11  VRDeck        6806 non-null   float64
 12  Name          6788 non-null   object 
dtypes: float64(6), object(7)
memory usage: 760.6+ KB


check the percentage of null values on each column.

In [11]:
X_train.isnull().sum()/len(X_train)

PassengerId     0.000000
HomePlanet      0.023152
CryoSleep       0.024446
Cabin           0.022433
Destination     0.019988
Age             0.020564
VIP             0.023008
RoomService     0.020851
FoodCourt       0.020995
ShoppingMall    0.023440
Spa             0.020708
VRDeck          0.021283
Name            0.023871
dtype: float64

Check the number of unique items in Cabin column. it is 0.77 that is high percentage. we decide to drop it

In [18]:
X_train['Cabin'].nunique()/len(X_train)

0.7776819096922635

Let's delete the Cabin  PassengerId/ name/ columns for simplicity

In [19]:
X_train.drop(['PassengerId','Cabin','Name'],axis=1,inplace=True)
X_train

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
1085,Mars,False,TRAPPIST-1e,41.0,False,29.0,0.0,1095.0,0.0,4.0
5656,Europa,False,55 Cancri e,25.0,False,140.0,4451.0,0.0,0.0,2050.0
5173,Earth,False,PSO J318.5-22,29.0,False,0.0,0.0,20.0,0.0,402.0
6287,Europa,True,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,NaN
3512,Earth,False,PSO J318.5-22,22.0,False,2.0,257.0,348.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1564,Earth,False,TRAPPIST-1e,41.0,False,0.0,82.0,3.0,0.0,718.0
7149,Europa,False,55 Cancri e,63.0,False,0.0,5036.0,4.0,519.0,0.0
381,Europa,False,TRAPPIST-1e,NaN,False,8.0,3584.0,0.0,13.0,57.0
8535,Europa,True,55 Cancri e,38.0,False,0.0,0.0,0.0,0.0,0.0


Fill in missing value

In [22]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6954 entries, 1085 to 2037
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    6793 non-null   object 
 1   CryoSleep     6784 non-null   object 
 2   Destination   6815 non-null   object 
 3   Age           6811 non-null   float64
 4   VIP           6794 non-null   object 
 5   RoomService   6809 non-null   float64
 6   FoodCourt     6808 non-null   float64
 7   ShoppingMall  6791 non-null   float64
 8   Spa           6810 non-null   float64
 9   VRDeck        6806 non-null   float64
dtypes: float64(6), object(4)
memory usage: 855.6+ KB


In [23]:
float64_columns = ['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
object_columns = ['HomePlanet','CryoSleep','Destination','VIP']

In [24]:
from sklearn.impute  import SimpleImputer
import numpy as np

imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
imp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [25]:
imp_median.fit(X_train[float64_columns])
imp_mode.fit(X_train[object_columns])

SimpleImputer(strategy='most_frequent')

In [26]:
imp_median.feature_names_in_

array(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'],
      dtype=object)

In [27]:
imp_median.statistics_

array([27.,  0.,  0.,  0.,  0.,  0.])

In [28]:
X_train.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,6811.000000,6809.000000,6808.000000,6791.000000,6810.000000,6806.000000
mean,28.816327,224.768101,453.875734,174.498454,317.831718,310.086247
std,14.554042,656.223702,1561.008679,603.410043,1171.133142,1167.428051
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,58.000000,83.000000,30.500000,62.000000,49.000000
max,79.000000,14327.000000,27723.000000,23492.000000,22408.000000,24133.000000


In [33]:
imp_mode.feature_names_in_

array(['HomePlanet', 'CryoSleep', 'Destination', 'VIP'], dtype=object)

In [34]:
imp_mode.statistics_

array(['Earth', False, 'TRAPPIST-1e', False], dtype=object)

In [38]:
X_train[float64_columns] = imp_median.transform(X_train[float64_columns])

In [39]:
X_train[object_columns] = imp_mode.transform(X_train[object_columns])

In [40]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6954 entries, 1085 to 2037
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    6954 non-null   object 
 1   CryoSleep     6954 non-null   object 
 2   Destination   6954 non-null   object 
 3   Age           6954 non-null   float64
 4   VIP           6954 non-null   object 
 5   RoomService   6954 non-null   float64
 6   FoodCourt     6954 non-null   float64
 7   ShoppingMall  6954 non-null   float64
 8   Spa           6954 non-null   float64
 9   VRDeck        6954 non-null   float64
dtypes: float64(6), object(4)
memory usage: 855.6+ KB


Lets encode the data

In [42]:
X_train[object_columns]

,HomePlanet,CryoSleep,Destination,VIP
1085,Mars,False,TRAPPIST-1e,False
5656,Europa,False,55 Cancri e,False
5173,Earth,False,PSO J318.5-22,False
6287,Europa,True,TRAPPIST-1e,False
3512,Earth,False,PSO J318.5-22,False
...,...,...,...,...
1564,Earth,False,TRAPPIST-1e,False
7149,Europa,False,55 Cancri e,False
381,Europa,False,TRAPPIST-1e,False
8535,Europa,True,55 Cancri e,False


In [45]:
for col in object_columns:
  print(col,": ",X_train[col].unique())

HomePlanet :  ['Mars' 'Europa' 'Earth']
CryoSleep :  [False True]
Destination :  ['TRAPPIST-1e' '55 Cancri e' 'PSO J318.5-22']
VIP :  [False True]


In [46]:
ordinal_columns = ['CryoSleep','VIP']
nominal_columns = ['HomePlanet','Destination']

In [47]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_enc = OrdinalEncoder()


In [48]:
ordinal_enc.fit(X_train[ordinal_columns])

OrdinalEncoder()

In [49]:
ordinal_enc.feature_names_in_

array(['CryoSleep', 'VIP'], dtype=object)

In [50]:
ordinal_enc.categories_

[array([False, True], dtype=object), array([False, True], dtype=object)]

In [51]:
X_train[ordinal_columns] = ordinal_enc.transform(X_train[ordinal_columns])
X_train[ordinal_columns]

,CryoSleep,VIP
1085,0.0,0.0
5656,0.0,0.0
5173,0.0,0.0
6287,1.0,0.0
3512,0.0,0.0
...,...,...
1564,0.0,0.0
7149,0.0,0.0
381,0.0,0.0
8535,1.0,0.0


In [53]:
from sklearn.preprocessing import OneHotEncoder
onehot_enc = OneHotEncoder(handle_unknown='ignore')

onehot_enc.fit(X_train[nominal_columns])

OneHotEncoder(handle_unknown='ignore')

In [54]:
onehot_enc.feature_names_in_

array(['HomePlanet', 'Destination'], dtype=object)

In [56]:
onehot_enc.categories_

[array(['Earth', 'Europa', 'Mars'], dtype=object),
 array(['55 Cancri e', 'PSO J318.5-22', 'TRAPPIST-1e'], dtype=object)]

In [55]:
onehot_enc.get_feature_names_out()

array(['HomePlanet_Earth', 'HomePlanet_Europa', 'HomePlanet_Mars',
       'Destination_55 Cancri e', 'Destination_PSO J318.5-22',
       'Destination_TRAPPIST-1e'], dtype=object)

In [58]:
encoded_nominal_columns = onehot_enc.get_feature_names_out()
encoded_nominal_columns

array(['HomePlanet_Earth', 'HomePlanet_Europa', 'HomePlanet_Mars',
       'Destination_55 Cancri e', 'Destination_PSO J318.5-22',
       'Destination_TRAPPIST-1e'], dtype=object)

In [65]:
X_train[encoded_nominal_columns] = onehot_enc.transform(X_train[nominal_columns]).toarray()
X_train[encoded_nominal_columns]

,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
1085,0.0,0.0,1.0,0.0,0.0,1.0
5656,0.0,1.0,0.0,1.0,0.0,0.0
5173,1.0,0.0,0.0,0.0,1.0,0.0
6287,0.0,1.0,0.0,0.0,0.0,1.0
3512,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...
1564,1.0,0.0,0.0,0.0,0.0,1.0
7149,0.0,1.0,0.0,1.0,0.0,0.0
381,0.0,1.0,0.0,0.0,0.0,1.0
8535,0.0,1.0,0.0,1.0,0.0,0.0


In [66]:
X_train

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
1085,Mars,0.0,TRAPPIST-1e,41.0,0.0,29.0,0.0,1095.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,1.0
5656,Europa,0.0,55 Cancri e,25.0,0.0,140.0,4451.0,0.0,0.0,2050.0,0.0,1.0,0.0,1.0,0.0,0.0
5173,Earth,0.0,PSO J318.5-22,29.0,0.0,0.0,0.0,20.0,0.0,402.0,1.0,0.0,0.0,0.0,1.0,0.0
6287,Europa,1.0,TRAPPIST-1e,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3512,Earth,0.0,PSO J318.5-22,22.0,0.0,2.0,257.0,348.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1564,Earth,0.0,TRAPPIST-1e,41.0,0.0,0.0,82.0,3.0,0.0,718.0,1.0,0.0,0.0,0.0,0.0,1.0
7149,Europa,0.0,55 Cancri e,63.0,0.0,0.0,5036.0,4.0,519.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
381,Europa,0.0,TRAPPIST-1e,27.0,0.0,8.0,3584.0,0.0,13.0,57.0,0.0,1.0,0.0,0.0,0.0,1.0
8535,Europa,1.0,55 Cancri e,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [68]:
X_train = X_train.drop(nominal_columns,axis=1)

In [69]:
X_train

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
1085,0.0,41.0,0.0,29.0,0.0,1095.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,1.0
5656,0.0,25.0,0.0,140.0,4451.0,0.0,0.0,2050.0,0.0,1.0,0.0,1.0,0.0,0.0
5173,0.0,29.0,0.0,0.0,0.0,20.0,0.0,402.0,1.0,0.0,0.0,0.0,1.0,0.0
6287,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3512,0.0,22.0,0.0,2.0,257.0,348.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1564,0.0,41.0,0.0,0.0,82.0,3.0,0.0,718.0,1.0,0.0,0.0,0.0,0.0,1.0
7149,0.0,63.0,0.0,0.0,5036.0,4.0,519.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
381,0.0,27.0,0.0,8.0,3584.0,0.0,13.0,57.0,0.0,1.0,0.0,0.0,0.0,1.0
8535,1.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [70]:
y_train

1085    False
5656     True
5173    False
6287     True
3512     True
        ...  
1564    False
7149     True
381      True
8535     True
2037    False
Name: Transported, Length: 6954, dtype: bool

Let's encode target column y

In [71]:
from sklearn.preprocessing import LabelEncoder
label_enc = LabelEncoder()

label_enc.fit(y_train)

LabelEncoder()

In [73]:
label_enc.classes_

array([False,  True])

In [74]:
y_train = label_enc.transform(y_train)
y_train

array([0, 1, 0, ..., 1, 1, 0])

In [75]:
y_train.shape

(6954,)

Create and train the model XGBoost

In [76]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

xgb = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed=42
)

params_xgb = {
    'max_depth': range (2, 10, 1),
    'n_estimators': range(100, 210, 10),
    'learning_rate': [0.1, 0.01, 0.05]
}

gs_xgb = GridSearchCV(estimator=xgb , param_grid=params_xgb, cv=5, verbose=3, scoring='accuracy')

In [77]:
gs_xgb.fit(X_train,y_train)

Fitting 5 folds for each of 264 candidates, totalling 1320 fits
[CV 1/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=0.788 total time=   0.6s
[CV 2/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=0.781 total time=   0.3s
[CV 3/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=0.797 total time=   0.3s
[CV 4/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=0.793 total time=   0.4s
[CV 5/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=0.794 total time=   0.4s
[CV 1/5] END learning_rate=0.1, max_depth=2, n_estimators=110;, score=0.788 total time=   0.3s
[CV 2/5] END learning_rate=0.1, max_depth=2, n_estimators=110;, score=0.781 total time=   0.3s
[CV 3/5] END learning_rate=0.1, max_depth=2, n_estimators=110;, score=0.797 total time=   0.3s
[CV 4/5] END learning_rate=0.1, max_depth=2, n_estimators=110;, score=0.794 total time=   0.3s
[CV 5/5] END learning_rate=0.1, max_depth=2, n_estimators=110;, score=0.795 total

GridSearchCV(cv=5, estimator=XGBClassifier(nthread=4, seed=42),
             param_grid={'learning_rate': [0.1, 0.01, 0.05],
                         'max_depth': range(2, 10),
                         'n_estimators': range(100, 210, 10)},
             scoring='accuracy', verbose=3)

In [78]:
gs_xgb.best_params_

{'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 110}

In [79]:
gs_xgb.best_score_

0.7992536811672156

do the feature engineering process to X_test

In [80]:
X_test

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
3745,4006_02,Earth,True,G/660/S,TRAPPIST-1e,2.0,False,0.0,0.0,0.0,0.0,0.0,Becca Wolfaddox
8501,9076_01,Earth,NaN,G/1461/S,TRAPPIST-1e,32.0,False,0.0,0.0,0.0,0.0,0.0,Shawne Simonsents
5913,6271_01,Europa,False,A/62/P,TRAPPIST-1e,37.0,False,0.0,0.0,0.0,0.0,0.0,Cheleon Wrempeedly
3382,3641_01,Earth,True,G/595/P,PSO J318.5-22,23.0,False,0.0,0.0,0.0,0.0,0.0,Erick Pacerty
6257,6623_01,NaN,True,B/218/P,55 Cancri e,38.0,False,NaN,0.0,0.0,0.0,0.0,Saxinon Geraing
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7755,8275_01,Mars,True,F/1588/S,TRAPPIST-1e,32.0,False,0.0,0.0,0.0,0.0,0.0,Slix Clake
1005,1075_01,Europa,True,B/41/S,TRAPPIST-1e,14.0,False,0.0,0.0,0.0,0.0,0.0,Teron Objeciane
2231,2390_01,Earth,NaN,G/385/P,TRAPPIST-1e,24.0,False,0.0,0.0,NaN,59.0,4120.0,Joelan Burrilllins
6800,7184_01,Earth,False,G/1170/S,TRAPPIST-1e,18.0,False,3.0,688.0,0.0,0.0,23.0,Eriana Barnoldeney


In [81]:
X_test.drop(['PassengerId','Cabin','Name'],axis=1,inplace=True)

X_test[float64_columns] = imp_median.transform(X_test[float64_columns])
X_test[object_columns] = imp_mode.transform(X_test[object_columns])

X_test[ordinal_columns] = ordinal_enc.transform(X_test[ordinal_columns])
X_test[encoded_nominal_columns] = onehot_enc.transform(X_test[nominal_columns]).toarray()

X_test = X_test.drop(nominal_columns,axis=1)
X_test

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
3745,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
8501,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
5913,0.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3382,1.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
6257,1.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7755,1.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1005,1.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2231,0.0,24.0,0.0,0.0,0.0,0.0,59.0,4120.0,1.0,0.0,0.0,0.0,0.0,1.0
6800,0.0,18.0,0.0,3.0,688.0,0.0,0.0,23.0,1.0,0.0,0.0,0.0,0.0,1.0


In [82]:
y_test

3745     True
8501    False
5913     True
3382     True
6257     True
        ...  
7755     True
1005     True
2231    False
6800     True
4168     True
Name: Transported, Length: 1739, dtype: bool

In [83]:
y_test = label_enc.transform(y_test)
y_test

array([1, 0, 1, ..., 0, 1, 1])

Let's see the accuracy score on test set

In [84]:
from sklearn.metrics import accuracy_score 
accuracy_score(y_test,gs_xgb.predict(X_test))

0.78953421506613

Load the Kaggle test dataset

In [85]:
import pandas as pd
X_realworld = pd.read_csv('/content/test.csv')

X_realworld

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


In [86]:
X_realworld.drop(['PassengerId','Cabin','Name'],axis=1,inplace=True)

X_realworld[float64_columns] = imp_median.transform(X_realworld[float64_columns])
X_realworld[object_columns] = imp_mode.transform(X_realworld[object_columns])

X_realworld[ordinal_columns] = ordinal_enc.transform(X_realworld[ordinal_columns])
X_realworld[encoded_nominal_columns] = onehot_enc.transform(X_realworld[nominal_columns]).toarray()

X_realworld = X_realworld.drop(nominal_columns,axis=1)
X_realworld

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,1.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.0,19.0,0.0,0.0,9.0,0.0,2823.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,1.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0.0,38.0,0.0,0.0,6652.0,0.0,181.0,585.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0,20.0,0.0,10.0,0.0,635.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4273,0.0,42.0,0.0,0.0,847.0,17.0,10.0,144.0,1.0,0.0,0.0,0.0,0.0,1.0
4274,1.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4275,0.0,27.0,0.0,0.0,2680.0,0.0,0.0,523.0,0.0,1.0,0.0,0.0,0.0,1.0


In [88]:
gender_submission = pd.read_csv('/content/sample_submission.csv')

In [89]:
gender_submission

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False
...,...,...
4272,9266_02,False
4273,9269_01,False
4274,9271_01,False
4275,9273_01,False


In [90]:
gs_xgb_submission = gender_submission.copy()

Use our model to predict the Kaggle test data

In [91]:
y_pred = gs_xgb.predict(X_realworld)
label_enc.inverse_transform(y_pred)

array([ True, False,  True, ...,  True,  True, False])

inverse transform y 

In [92]:
y_pred = label_enc.inverse_transform(y_pred)

In [93]:
gs_xgb_submission['Transported'] =  y_pred
gs_xgb_submission

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [94]:
gs_xgb_submission.to_csv('/content/gs_xgb_submission.csv',index=False)

accuracy is 0.79518 on Kaggle board 

In [98]:
X_train_test = pd.concat([X_train, X_test], ignore_index=False, sort=False)


In [102]:
y_train_test = np.concatenate([y_train,y_test])

In [103]:
y_train_test

array([0, 1, 0, ..., 0, 1, 1])

In [104]:
xgb_best_setting = XGBClassifier(
    objective = 'binary:logistic',
    nthread = 4,
    seed = 42,
    learning_rate = 0.05,
    max_depth = 5,
    n_estimators = 110
) 

In [106]:
xgb_best_setting.fit(X_train_test,y_train_test)

XGBClassifier(learning_rate=0.05, max_depth=5, n_estimators=110, nthread=4,
              seed=42)

In [107]:
gs_xgb_merged_train_test_submission = gender_submission.copy()

gs_xgb_merged_train_test_submission['Transported'] = label_enc.inverse_transform(gs_xgb.predict(X_realworld))

In [108]:
gs_xgb_merged_train_test_submission.to_csv('/content/gs_xgb_merged_train_test_submission.csv',index=False)

accuracy is 0.79518 on Kaggle board. same as previous model, maybe we have to dive into the feature engineering part that we dicided to drop some columns at the very beginning ['PassengerId','Cabin','Name'] the useful information may hide in there.
